## Script para automatizar datos 

Recuerda poner tus datos en la misma carpeta del script y cambiar la variable "ruta_csv" por el nombre de tu archivo

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import os

# 1. Carga de Datos
ruta_csv = "Coordenadas_Predio.csv"
df = pd.read_csv(ruta_csv)

# 2. Geometría
# Se agrupan las columnas X e Y en una lista de tuplas (pares de coordenadas)
lista_vertices = list(zip(df['Este_X'], df['Norte_Y']))
geometria_poligono = Polygon(lista_vertices)

# 3. Creación del GeoDataFrame
gdf = gpd.GeoDataFrame(
    {'ID': [1], 'Nombre': ['Fundo_Santa_Rosa_Irregular']}, 
    geometry=[geometria_poligono]
)

# 4. Georreferenciación
# Se define explícitamente el sistema WGS 84 / UTM zone 18S
gdf.set_crs(epsg=32718, inplace=True)

# 5. Cálculo de Atributos
# Área en Hectáreas y Perímetro en Kilómetros
gdf['Area_Ha'] = round(gdf.area / 10000, 4)
gdf['Perim_Km'] = round(gdf.length / 1000, 4)

# Funcionalidad Extra: Centroide (útil para etiquetas en planos)
gdf['Centro_X'] = round(gdf.centroid.x, 2)
gdf['Centro_Y'] = round(gdf.centroid.y, 2)

# 6. Exportar
carpeta_salida = "Resultado_GIS"
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)

ruta_shp = f"{carpeta_salida}/Predio_Final.shp"
gdf.to_file(ruta_shp)

print(f"Shapefile guardado en: {ruta_shp}")
print(gdf.drop(columns='geometry'))